In [28]:
import datetime
import pandas as pd
import pickle
import string
import re

from collections import defaultdict, Counter
from os import path

## Database
import pymongo
from pymongo import MongoClient

In [29]:
def pickleme(temp, filename):
    
    with open(filename + '.pkl', 'w') as picklefile:
        pickle.dump(temp, picklefile)
    
    return

In [30]:
def getpickle(filename):

    try:
        with open(filename + '.pkl', 'rb') as picklefile:
            return pickle.load(picklefile)
    except:
        return 'There was an error trying to read this file.  Please check the filename or path.'

In [84]:
def cleantext(temp):
    
    badchar = [('*', ' '), ('/', ' '), ('- ', ' '), ('--', ' '),('\r', ' '), ('\t', ' '), ('\n', ' '), ('\x8e', 'e'), 
               ('\xe2', ' '), ('\x80', ' '), ('\xca', ' '), ('\xc9', ' '), ('\xd0', ' '), ('\xd2', ' '), 
               ('\xd3', ' '), ('\xd4', ' '), ('\xd5', ' '), ('\xa2', ' '), ('\xa5', ' '), ('\xd1', ' '), 
               ('\x93', ' '), ('\x94', ' '), ('\x99', ' '), ('\xff', ' '), ('\x82', 'e'), ('\x0c', ' '), 
               ('\xa0', ' '), ('\xc2', ' '), ('\xc3\xa9', 'e'), ('\x9d', ' '), ('\x9c', ' '), ('\x98', ' ')]
    
    for bad in badchar:
        temp = temp.replace(bad[0], bad[1])

    return temp

Connect to the mongoDB server with mongo.
```
help
show dbs
use new_cool_db
show dbs
```

In [42]:
! brew services start mongodb

==> Successfully started `mongodb` (label: homebrew.mxcl.mongodb)


In [43]:
# ! brew services stop mongodb

#### Read, clean, tag and load resume corpus into list of dictionaries

In [85]:
end = 73
path = 'dataResumes/'

## Initialize MongoDB for use
client = MongoClient()
db = client['resume_db']
resumes = db.resume_collection

## Clear out the database from any residual records from prior code testing
resumes.drop()

for i in range(end):
    filename = '0' * (3 - len(str(i + 1))) + str(i + 1)
    f = open(path + filename + '.txt', 'r')
    
    resumes.insert_one({ 'Id': filename,
                     'Resume': cleantext(f.read())
                   })

In [11]:
# ## Initialize MongoDB for use
# client = MongoClient()
# db = client['resume_db']
# resumes = db.resume_collection

In [81]:
print "Count of number of documents:", resumes.count()
print "Keys in one example document:", resumes.find_one().keys()
# resumes.find_one() # Uncomment this line to see what a full document looks like.

Count of number of documents: 73
Keys in one example document: [u'_id', u'Id', u'Resume']


In [82]:
corpus = []

for resume in resumes.find({}, {'Resume' : 1, "_id": 0}):
    corpus.append(resume['Resume'])

print len(corpus)

73


In [83]:
corpus[1]

u'EDUCATION The George Washington University, Elliott School of International Affairs, Washington, DC Master of Arts Candidate in Middle East Studies; GPA: 3.97         08 14  The Hebrew University of Jerusalem, Jerusalem, Israel Bachelor of Arts in Islamic and Middle Eastern Studies and International Affairs, Cum Laude     12 10  EXPERIENCE Filter Coffeehouse, Washington, DC                       10 12 present  The NGO Committee on the Status of Women, New York, NY         09 11 05 12 Intern   Coordinated preparations and execution of parallel events for the 56th annual UN Convention on the Status of Women.    Managed and developed relations with members representing approximately 80 UN accredited Non-Governmental Organizations.   Participated in day-to-day office operation including answering phones, handling email correspondences, updating social media forums, database keeping, event planning, etc.  Think Coffee, New York, NY                  01 11 05 12  Center for Strategic and In